In [ ]:
# import libraries
import BioSimSpace as BSS
import sys
import os
from sklearn.metrics import mean_absolute_error as MAE

if '/home/anna/Documents/cinnabar' not in sys.path:
    sys.path.insert(1, '/home/anna/Documents/cinnabar')
import cinnabar

print("adding code to the pythonpath...")
code = '/home/anna/Documents/code/python'
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

print(cinnabar.__file__)

from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *

In [ ]:
bench_folder = f"/home/anna/Documents/benchmark"
protein = "tyk2"
main_dir = f"{bench_folder}/extracted/{protein}"

# choose location for the files
net_file = f"{main_dir}/execution_model/network_lomap.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
exp_file = f"{bench_folder}/inputs/experimental/{protein}.yml"

if os.path.exists(f"{main_dir}/outputs_extracted/results"):
    results_folder = f"{main_dir}/outputs_extracted/results"
elif os.path.exists(f"{main_dir}/outputs/results"):
    results_folder = f"{main_dir}/outputs/results"
else:
    raise ValueError(f"results directory not found in the {main_dir}. please make sure results were written using the analysis script previously in the pipeline")

output_folder = validate.folder_path(f"{main_dir}/analysis", create=True)

all_analysis_object = analysis_network(results_folder,
                                       exp_file=exp_file,
                                       net_file=net_file,
                                       output_folder=output_folder,
                                       analysis_ext=ana_file
                                        )

# can add any other results files
# all_analysis_object.compute_other_results(file_name=None, name=None)
all_analysis_object.compute(cycle_closure=True)

In [ ]:
all_analysis_object._add_fwf_path('/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis')

eng = "SOMD"

# get the experimental dictionary
exp_dicts = all_analysis_object._get_exp_fwf()
for key in exp_dicts[0]:
    print(f"{key} : {exp_dicts[0][key][0]}, {exp_dicts[0][key][1]}")

# get the network analysis
fwf_dict = all_analysis_object._get_ana_fwf(engine=eng)
for key in fwf_dict:
    print(f"{key} : {fwf_dict[key][0]}, {fwf_dict[key][1]}")

dict_y = fwf_dict
dict_x = all_analysis_object.cinnabar_calc_val_dict[eng]
dict_exp = exp_dicts[0]

df1 = plotting_engines.match_dicts_to_df(dict_x, dict_y, "cinnabar", "fwf")
df2 = plotting_engines.match_dicts_to_df(dict_exp, dict_x, "experimental", "cinnabar").drop(columns=["freenrg_cinnabar", "err_cinnabar"])
df3 = df1.join(df2)
df3


In [ ]:
df3.fillna(0)
df3.plot.bar(y=["freenrg_fwf","freenrg_cinnabar","freenrg_experimental"],
            yerr=df3[["err_fwf","err_cinnabar","err_experimental"]].T.values,
            title=f"fwf, cinnabar, experimental, {eng}",
            xlabel="ligands",
            ylabel="dG (kcal/mol)")
df3.dropna()
# df["values"] = df.index

print("cinnabar")

df3.plot.scatter(
        x="freenrg_experimental",y="freenrg_cinnabar",
        xerr="err_experimental",
        yerr="err_cinnabar",
        title=f"cinnabar, {eng}",
        xlabel="experimental dG (kcal/mol)",
        ylabel="cinnabar dG (kcal/mol)")

# cinnabar correlation
print("mae")
c_mae = MAE(df3["freenrg_experimental"],df3["freenrg_cinnabar"])
print(c_mae)
print("mue")
print(all_analysis_object._stats_object.compute_mue(pert_val="val", y=eng))
c_mae_2 = all_analysis_object._stats_object._compute_stats(x=df3["freenrg_experimental"],
                                                 y=df3["freenrg_cinnabar"],
                                                 xerr=df3["err_experimental"],
                                                 yerr=df3["err_cinnabar"],
                                                 statistic="MUE")
print(c_mae_2)
print("ktau")
print(all_analysis_object._stats_object.compute_ktau(pert_val="val", y=eng))

print("fwf")

df3.plot.scatter(
        x="freenrg_experimental",y="freenrg_fwf",
        xerr="err_experimental",
        yerr="err_fwf",
        title=f"fwf, {eng}",
        xlabel="experimental dG (kcal/mol)",
        ylabel="fwf dG (kcal/mol)")


# fwf correlation
fwf_stats = all_analysis_object._get_stats_fwf(engine=eng)

print("mae")
f_mae = MAE(df3["freenrg_experimental"],df3["freenrg_fwf"])
print(f_mae)

# calculating using the cinnabar stats
f_mae_2 = all_analysis_object._stats_object._compute_stats(x=df3["freenrg_experimental"],
                                                 y=df3["freenrg_fwf"],
                                                 xerr=df3["err_experimental"],
                                                 yerr=df3["err_fwf"],
                                                 statistic="MUE")
print(f_mae_2)

print("between both net ana methods")

df3.plot.scatter(
        x="freenrg_cinnabar",y="freenrg_fwf",
        xerr="err_cinnabar",
        yerr="err_fwf",
        title=f"fwf, {eng}",
        xlabel="cinnabar dG (kcal/mol)",
        ylabel="fwf dG (kcal/mol)")

b_mae = all_analysis_object._stats_object._compute_stats(x=df3["freenrg_cinnabar"],
                                                 y=df3["freenrg_fwf"],
                                                 xerr=df3["err_cinnabar"],
                                                 yerr=df3["err_fwf"],
                                                 statistic="MUE")
print(b_mae)